### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 10, 2022

In [1]:
# todo: some of the 12-19c outputs are dupes/fucked ?

In [2]:
import numpy as np
import pandas as pd

In [3]:
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], axis= 1)

In [4]:
df = pd.read_csv(f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={0.01}", skiprows=1)
df = drop_standard_columns(df)
df.rename(columns={'Date and Time': 'date', 'annualised consumption GUESS, TWh': 0.01}, inplace=True)

In [5]:
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    tmp_df = pd.read_csv(f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", skiprows=1)
    tmp_df = drop_standard_columns(tmp_df)
    tmp_df.rename(columns={'Date and Time': 'date', 'annualised consumption GUESS, TWh': elec_str}, inplace=True)

    df = df.merge(tmp_df, how="inner", on="date")

In [6]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4493,2022-11-05T00:00:00,163.866323,143.583601,134.285053,110.768815,101.209223,91.810619,85.588362,76.682602,72.482848,67.336436,60.984880,55.207365,55.207365,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4494,2022-11-06T00:00:00,163.971725,143.583406,134.881514,110.986422,101.408051,92.172474,86.048285,76.833247,72.625242,67.988409,61.104686,55.315821,55.315821,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4495,2022-11-07T00:00:00,159.264941,139.461865,131.379610,108.093303,98.497148,89.722281,83.911211,74.627760,70.540545,66.614607,59.350685,53.727989,53.727989,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4496,2022-11-08T00:00:00,162.267071,142.090709,134.232157,110.125335,100.353810,91.622033,85.818192,76.034486,71.870228,68.457773,60.469440,54.740756,54.740756,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4497,2022-11-09T00:00:00,166.559653,145.849549,137.744846,113.030825,103.008551,94.030246,88.047489,78.045888,73.771470,70.208966,62.069087,56.188858,56.188858,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663


In [7]:
df.to_csv('./datasets/cbeci-annualized-consumption.csv', index = False)